# colab

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/bias gpt")
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [2]:
!pip install datasets
!pip install transformers==4.43

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.7 MB/s eta 0:00:00
   ━

# Head

In [63]:
import torch, importlib
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import random_split
import tensorflow as tf


model_name = "distilgpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 原神

In [68]:
prompt = "\n"
sentence = prompt + open('Genshin Impact.txt').read().replace('\n', ' ')
gpt_inputs = tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True)

In [69]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
    TextVAE.gpt.cuda()
tvae.train()
epochs = 100
optim = AdamW(tvae.parameters(), 5e-4)
scheduler = CosineAnnealingLR(optim, T_max = epochs, eta_min = 0)
for epoch in range(epochs):
    loss = tvae(**gpt_inputs)[0]
    loss.backward()
    optim.step()
    optim.zero_grad()
    scheduler.step()
    print(f"epoch: {epoch + 1}; loss: {loss.item()}")

4.1567606925964355
epoch: 1; loss: 4.1567606925964355
6.949851989746094
epoch: 2; loss: 6.949851989746094
4.8893141746521
epoch: 3; loss: 4.8893141746521
3.708163261413574
epoch: 4; loss: 3.708163261413574
3.2473034858703613
epoch: 5; loss: 3.2473034858703613
2.740238904953003
epoch: 6; loss: 2.740238904953003
2.601212501525879
epoch: 7; loss: 2.601212501525879
2.3769290447235107
epoch: 8; loss: 2.3769290447235107
2.176826238632202
epoch: 9; loss: 2.176826238632202
2.086315631866455
epoch: 10; loss: 2.086315631866455
1.927165150642395
epoch: 11; loss: 1.927165150642395
1.8198424577713013
epoch: 12; loss: 1.8198424577713013
1.7395508289337158
epoch: 13; loss: 1.7395508289337158
1.644917368888855
epoch: 14; loss: 1.644917368888855
1.5660734176635742
epoch: 15; loss: 1.5660734176635742
1.48930823802948
epoch: 16; loss: 1.48930823802948
1.4514402151107788
epoch: 17; loss: 1.4514402151107788
1.3891966342926025
epoch: 18; loss: 1.3891966342926025
1.3447506427764893
epoch: 19; loss: 1.3447506

tensor([[-0.5726, -0.1920,  0.1097,  ..., -1.2258, -1.3441, -1.2467]])
tensor([[[-22.7622, -20.7893, -22.7838,  ..., -34.2880, -33.8851, -22.2164],
         [-39.5813, -38.7599, -39.0928,  ..., -50.1195, -47.5210, -40.0836],
         [-43.3257, -42.9877, -45.0067,  ..., -51.9786, -49.5019, -43.1713],
         ...,
         [-46.6427, -47.2990, -50.0125,  ..., -56.6759, -56.4586, -48.5660],
         [-45.2951, -44.5778, -49.3268,  ..., -55.5477, -55.5071, -44.3952],
         [-44.6097, -46.7685, -48.5046,  ..., -55.1764, -54.4472, -46.8224]]])
3.570312261581421


In [ ]:
logits = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = False)[2]
print(tokenizer.decode(logits.argmax(-1)[0]))
print((logits.argmax(-1)[0] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
tvae.eval()
bias = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = True)[1]
print(bias.shape)
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2')
if torch.cuda.is_available():
    gpt = gpt.cuda()
with torch.no_grad():
    gpt.transformer.h[-1].mlp.c_fc.bias += bias[0]

In [ ]:
with torch.no_grad():
    tokens = gpt(gpt_inputs['input_ids'])[0][0].argmax(-1)
print(tokenizer.decode(tokens))
print((tokens[:-1] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
test_ids = tokenizer(prompt, return_tensors="pt").input_ids
if torch.cuda.is_available():
   test_ids = test_ids.to('cuda')

generated_ids = gpt.generate(
    test_ids,
    max_length = 128,
    num_return_sequences = 1,
    no_repeat_ngram_size = 5,
    num_beams = 5,
    eos_token_id = 50256,
    bad_words_ids = [[198]],
)

# 解码生成的文本
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text[0])
print(len(generated_ids[0]))

# wiki

In [41]:
wiki = load_dataset('wikipedia', '20220301.simple',
          trust_remote_code = True)["train"].select(range(1))

In [66]:
def mapping(x):
  gpt_text = ['\n' + text.replace('\n',' ') for text in x['text']]
  return tokenizer(gpt_text, return_tensors="pt", padding = 'max_length',
                       max_length = 128, truncation = True)

dataset = wiki.map(mapping, batched = True, load_from_cache_file = True,
                   remove_columns=['id','url','title','text'])
dataset.set_format('torch')
print(dataset)
train_size = len(dataset) - len(dataset) // 20
trainData, evalData = random_split(dataset, [train_size, len(dataset) // 20])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1
})


c:\Users\fudan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [67]:
tokenizer.decode(trainData[0]['input_ids'])

"\nApril is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.  April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.  April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.  The Month   April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four"

In [51]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
    TextVAE.gpt.cuda()

In [ ]:
tokenizer.batch_decode(tvae(**trainData[0:3])[2].argmax(-1))

In [52]:
training_args = TrainingArguments(
    output_dir = "./results",
    save_strategy = "no",
    num_train_epochs = 100,
    weight_decay = 0,
    learning_rate = 5e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    warmup_steps = 10,
    logging_steps = 32,
    # eval_strategy= "steps",
    # eval_steps = 10,
    # fp16 = True,
    lr_scheduler_type = 'linear',
    max_grad_norm  = 1,
)

trainer = Trainer(
  model = tvae,
  args = training_args,
  train_dataset = trainData,
  # eval_dataset = evalData,
)
trainer.train()

  0%|          | 0/100 [00:00<?, ?it/s]

4.728677749633789
4.728677749633789
4.490350723266602
3.6916439533233643
3.0866031646728516
2.7311201095581055
2.650399684906006
2.592966318130493
2.202491521835327
1.9992694854736328
2.037381887435913
1.7427986860275269
1.6756482124328613
1.5219002962112427
1.5047425031661987
1.4837645292282104
1.3475985527038574
1.3288016319274902
1.3389934301376343
1.2478845119476318
1.2994017601013184
1.2180871963500977
1.2346391677856445
1.1994527578353882
1.1269956827163696
1.1093289852142334
1.09656822681427
1.0613446235656738
1.0473536252975464
1.0335854291915894
1.0170999765396118
0.9966658353805542
{'loss': 1.9241, 'grad_norm': 1.3017029762268066, 'learning_rate': 0.00037777777777777777, 'epoch': 32.0}
0.991558313369751
0.9787073135375977
0.9609749913215637
0.9611866474151611
0.9445890784263611
0.9404472708702087
0.9309391379356384
0.9263143539428711
0.9148540496826172
0.920029878616333
0.9081777334213257
0.9079878330230713
0.9007881879806519
0.8965179324150085
0.8910121321678162
0.8816137909

TrainOutput(global_step=100, training_loss=1.1956758332252502, metrics={'train_runtime': 128.1986, 'train_samples_per_second': 0.78, 'train_steps_per_second': 0.78, 'total_flos': 0.0, 'train_loss': 1.1956758332252502, 'epoch': 100.0})

In [ ]:
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2').to('cuda')
input_ids = torch.tensor([[1,2,3]])
mask = torch.tensor([[1,1,0]])
gpt(input_ids, attention_mask = mask)[0] * mask.unsqueeze(-1)

In [ ]:
!python TextVAE.py

2024-11-16 06:29:34.124057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 06:29:34.143725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 06:29:34.149694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 06:29:35.414994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
0.0


# TF_TextVAE

In [ ]:
!python TF_trainer.py --size 1 --batch_size 1 --warmup 10 --epoch 200 --ratio 1 --validation_freq 10000

In [ ]:
!python TF_TextVAE.py

In [ ]:
!python TextVAE.py